In [ ]:
# Tratamentos e treino

import os
import numpy as np
import tensorflow as tf
import librosa
from sklearn.preprocessing import MinMaxScaler
from keras.src.models import Sequential
from keras.src.layers import LSTM, Dense
from keras.src.callbacks import EarlyStopping
from wakepy import keep
import av
import random

# Função para converter arquivos para .wav
def to_wav(in_path: str, out_path: str = None, sample_rate: int = 16000) -> str:
    if out_path is None:
        out_path = os.path.splitext(in_path)[0] + '.wav'
    with av.open(in_path) as in_container:
        in_stream = in_container.streams.audio[0]
        with av.open(out_path, 'w', 'wav') as out_container:
            out_stream = out_container.add_stream(
                'pcm_s16le',
                rate=sample_rate,
                layout='mono'
            )
            for frame in in_container.decode(in_stream):
                for packet in out_stream.encode(frame):
                    out_container.mux(packet)
    return out_path

# Função para extrair frequência fundamental
def extract_f0(wav_path, fmin=50, fmax=500):
    y, sr = librosa.load(wav_path, sr=None)
    f0 = librosa.yin(y, fmin=fmin, fmax=fmax, sr=sr)
    return f0

# Função para dividir os arquivos em grupos
def groups_split(dataset, n_groups=5, seed=324591):
    if seed is not None:
        random.seed(seed)  # Garante reprodutibilidade
    
    dataset = dataset.copy()  # Faz uma cópia para evitar modificar a original
    random.shuffle(dataset)  # Embaralha os dados
    
    # Calcula o tamanho de cada grupo
    split_size = len(dataset) // n_groups
    groups = []
    
    # Cria os grupos
    for i in range(n_groups):
        start_idx = i * split_size
        end_idx = start_idx + split_size if i < n_groups - 1 else len(dataset)  # Garante que o último grupo pega o restante
        groups.append(dataset[start_idx:end_idx])
    
    return groups

# Diretório contendo as músicas
diretorio_musicas = r"D:/TCC/VH/VH/bd"
diretorio_wavs = r"D:/TCC/VH/VH/bd/wav"
diretorio_checkpoints = r"D:/TCC/VH/VH/bd/checkpoints"

# Garantir que os diretórios de saída existem
os.makedirs(diretorio_wavs, exist_ok=True)
os.makedirs(diretorio_checkpoints, exist_ok=True)

# Parâmetros do modelo
features = 1

# Criar o modelo LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(units=125, activation="tanh", input_shape=(None, features)))
model_lstm.add(Dense(units=1))
model_lstm.compile(optimizer="RMSprop", loss="mse")

# Callback para Early Stopping
callback = EarlyStopping(monitor='loss', patience=5, start_from_epoch=3)

# Contador de músicas
contador_musicas = 0
contador_cps = 0
contador_erros = 0

from keras.src.losses import mean_squared_error
from keras.src.saving.saving_api import load_model

# Verificar se existe um checkpoint para retomar o treinamento
checkpoint_path = sorted(
    [f for f in os.listdir(diretorio_checkpoints) if f.endswith('.keras')],
    key=lambda x: int(x.split('_')[-1].split('.')[0]),  # Extrai o número do nome do arquivo
    reverse=True  # Ordem decrescente
)

if checkpoint_path:
    # Carregar o modelo e o contador de músicas a partir do último checkpoint
    model_lstm = load_model(
        diretorio_checkpoints+checkpoint_path[0]
        # custom_objects={"mse": mean_squared_error}  # Mapear 'mse' para a função Keras correta
    )
    # Carregar o contador de músicas (assumindo que o nome do arquivo de checkpoint tem um formato que inclui o número de músicas processadas)
    checkpoint_idx = int(checkpoint_path[0].split('_')[-1].split('.')[0])
    contador_musicas = checkpoint_idx  # Continuar de onde parou
    print(f"Modelo carregado a partir do checkpoint: {checkpoint_path[0]}")
else:
    print("Nenhum checkpoint encontrado. Iniciando treinamento do início.")

# Listar todos os arquivos .mp3
arquivos_mp3 = []
for root, dirs, files in os.walk(diretorio_musicas):
    for file in files:
        if file.endswith(".mp3"):
            arquivos_mp3.append(os.path.join(root, file))

# Escolha o grupo de teste pelo índice (0 a n_groups-1)
grupo_teste_idx = 0  # Por exemplo, escolha o primeiro grupo como teste

# Dividir os arquivos em conjuntos de treino e teste
n_groups = 5  # Número de grupos (ajuste conforme necessário)
groups = groups_split(arquivos_mp3, n_groups)

# Separar os arquivos de treino e teste
test_files = groups[grupo_teste_idx]  # Usar o grupo escolhido como teste
train_files = sum([group for idx, group in enumerate(groups) if idx != grupo_teste_idx], [])  # Todos os outros como treino
print(len(train_files))
print(len(test_files))

with keep.presenting():
    # Processar cada música do conjunto de treino
    for idx, file in enumerate(train_files[contador_musicas:], start=contador_musicas):  # Iniciar do índice salvo
        wav_path = os.path.join(diretorio_wavs, os.path.splitext(os.path.basename(file))[0] + '.wav')

        try:
            # Converter para wav
            wav_path = to_wav(file, wav_path)

            # Extrair f0
            f0 = extract_f0(wav_path)

            # Normalizar os dados
            sc = MinMaxScaler(feature_range=(0, 1))
            f0 = f0.reshape(-1, 1)
            f0_scaled = sc.fit_transform(f0)

            # Adicionar uma dimensão para o LSTM
            f0_scaled = f0_scaled.reshape(1, f0_scaled.shape[0], features)

            # Treinar o modelo para esta música
            model_lstm.fit(
                f0_scaled, f0_scaled,
                epochs=20,
                batch_size=1,
                callbacks=[callback],
                verbose=1
            )

            # Incrementar o contador
            contador_musicas += 1

            # Salvar checkpoint a cada 100 músicas
            if contador_musicas % 100 == 0:
                checkpoint_path = os.path.join(diretorio_checkpoints, f"modelo_checkpoint_{contador_musicas}.keras")
                model_lstm.save(checkpoint_path)
                print(f"Checkpoint salvo: {checkpoint_path}")
                contador_cps = contador_cps + 1

        except Exception as e:
            # Caso haja erro, exibe uma mensagem e continua com o próximo arquivo
            print(f"Erro ao processar o arquivo {os.path.basename(file)}: {e}")
            contador_erros = contador_erros + 1
            continue
        
        print(f"Progresso Músicas: {contador_musicas}/{len(train_files)-contador_erros}")
        print(f"Progresso Checkpoints: {contador_cps}/64")
        print(f"Contagem de erros: {contador_erros}")

        

# Salvar o modelo final após processar todas as músicas
modelo_final_path = os.path.join(diretorio_checkpoints, "modelo_final.keras")
model_lstm.save(modelo_final_path)
print(f"Modelo final salvo: {modelo_final_path}")


c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Nenhum checkpoint encontrado. Iniciando treinamento do início.
6400
1600
Progresso Músicas: 1/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 2/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 3/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 4/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 5/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 6/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 7/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 8/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 9/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 10/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 11/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 12/6400
Progresso Checkpoints: 0/64
Contagem de erros: 0
Progresso Músicas: 13/64

In [15]:
# Separação dos grupos para validação por gênero

import pandas as pd
import re

def groups_split(dataset, n_groups=5, seed=324591):
    if seed is not None:
        np.random.seed(seed)  # Garante reprodutibilidade
    
    dataset = dataset.copy()  # Faz uma cópia para evitar modificar a original
    np.random.shuffle(dataset)  # Embaralha os dados
    
    # Calcula o tamanho de cada grupo
    split_size = len(dataset) // n_groups
    groups = []
    
    # Cria os grupos
    for i in range(n_groups):
        start_idx = i * split_size
        end_idx = start_idx + split_size if i < n_groups - 1 else len(dataset)  # Garante que o último grupo pega o restante
        groups.append(dataset[start_idx:end_idx])
    
    return groups

diretorio_musicas = r"D:/TCC/VH/VH/bd"
diretorio_wavs = r"D:/TCC/VH/VH/bd/wav"

# # Listar todos os arquivos .wav
arquivos_wav = []
for root, dirs, files in os.walk(diretorio_wavs):
    for file in files:
        if file.endswith(".wav"):
            arquivos_wav.append(os.path.join(root, file))

# # Escolha o grupo de teste pelo índice (0 a n_groups-1)
grupo_teste_idx = 0  # Por exemplo, escolha o primeiro grupo como teste

# # Dividir os arquivos em conjuntos de treino e teste
n_groups = 5  # Número de grupos (ajuste conforme necessário)
groups = groups_split(dataset=arquivos_wav, n_groups=n_groups)

df_generos = pd.read_csv("C:/Vitor Hugo/TCC/Teste/__pycache__/generos.csv")
df_teste = pd.DataFrame(groups[0], columns=["caminho"])
df_teste["id"] = df_teste["caminho"].apply(lambda x: re.search(r"(\d+)\.wav$", x).group(1) if re.search(r"(\d+)\.wav$", x) else None)
df_generos["id"] = df_generos["id"].astype(int)
df_teste["id"] = df_teste["id"].astype(int)
df_teste_categorizado = df_teste.merge(df_generos, on="id", how="left")
df_teste_categorizado.drop(df_teste_categorizado.columns[2], axis=1, inplace=True)
print(df_teste_categorizado)
# Agrupar as músicas por gênero e criar listas de caminhos
musicas_por_genero = df_teste_categorizado.groupby("genre")["caminho"].apply(list).to_dict()



                             caminho      id          genre
0     D:/TCC/VH/VH/bd/wav\107535.wav  107535   Instrumental
1     D:/TCC/VH/VH/bd/wav\122807.wav  122807            Pop
2     D:/TCC/VH/VH/bd/wav\137212.wav  137212           Rock
3     D:/TCC/VH/VH/bd/wav\148121.wav  148121   Instrumental
4     D:/TCC/VH/VH/bd/wav\024717.wav   24717  International
...                              ...     ...            ...
1594  D:/TCC/VH/VH/bd/wav\129674.wav  129674        Hip-Hop
1595  D:/TCC/VH/VH/bd/wav\036388.wav   36388            Pop
1596  D:/TCC/VH/VH/bd/wav\010671.wav   10671           Rock
1597  D:/TCC/VH/VH/bd/wav\086140.wav   86140   Experimental
1598  D:/TCC/VH/VH/bd/wav\060544.wav   60544  International

[1599 rows x 3 columns]


In [ ]:
# Validação por gênero
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import librosa
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
from keras.src.models import Sequential
from keras.src.layers import LSTM, Dense
from keras.src.callbacks import EarlyStopping
from keras.src.saving.saving_api import load_model
from wakepy import keep

# Configurações gerais
SAMPLE_RATE = 43  # Taxa de amostragem do Librosa
INTERVAL_DURATION = 10  # Cada intervalo tem 10 segundos
INPUT_DURATION = 5  # 5 segundos como entrada no modelo
PREDICTION_DURATION = 5  # 5 segundos para validar a predição
NUM_ITERATIONS = 10  # Número de iterações (repetições) por música

def groups_split(dataset, n_groups=5, seed=324591):
    if seed is not None:
        np.random.seed(seed)  # Garante reprodutibilidade
    
    dataset = dataset.copy()  # Faz uma cópia para evitar modificar a original
    np.random.shuffle(dataset)  # Embaralha os dados
    
    # Calcula o tamanho de cada grupo
    split_size = len(dataset) // n_groups
    groups = []
    
    # Cria os grupos
    for i in range(n_groups):
        start_idx = i * split_size
        end_idx = start_idx + split_size if i < n_groups - 1 else len(dataset)  # Garante que o último grupo pega o restante
        groups.append(dataset[start_idx:end_idx])
    
    return groups

# Função para carregar séries de áudio e calcular a frequência fundamental
def load_series(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    fmin = 50  # Frequência mínima esperada (em Hz)
    fmax = 500  # Frequência máxima esperada (em Hz)
    f0 = librosa.yin(y, fmin=fmin, fmax=fmax, sr=sr)
    return f0

# Função para dividir a série temporal em 5 segundos de treino e 5 segundos de teste
def get_train_test_segments(f0, start_point, input_duration=INPUT_DURATION, prediction_duration=PREDICTION_DURATION):
    # Extrair os 5 segundos de treino e 5 segundos de teste a partir do ponto de início
    start_train = start_point
    end_train = start_train + (input_duration * SAMPLE_RATE)
    start_test = end_train
    end_test = start_test + (prediction_duration * SAMPLE_RATE)
    
    # Garantir que os índices não ultrapassem o tamanho da série
    if end_test > len(f0):
        return None, None  # Se não houver dados suficientes, retorna None
    
    train_segment = f0[start_train:end_train]
    test_segment = f0[start_test:end_test]
    
    return train_segment, test_segment

# Função para calcular o RMSE Global
def calculate_rmse_global(predictions, targets):
    mse = np.mean((predictions - targets) ** 2)
    rmse = np.sqrt(mse)
    return rmse

def split_sequence(sequence, n_steps):
            X, y = list(), list()
            for i in range(len(sequence)):
                end_ix = i + n_steps
                if end_ix > len(sequence) - 1:
                    break
                seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
                X.append(seq_x)
                y.append(seq_y)
            return np.array(X), np.array(y)

model = load_model("C://Vitor Hugo//TCC//Teste//__pycache__//modelo_final.keras")

# Criar um DataFrame para armazenar os resultados dos RMSEs
results = pd.DataFrame(columns=["Gênero", "RMSE base", "RMSE ajustado", "MAPE"])

# Gerar uma lista de sementes aleatórias para garantir reprodutibilidade
np.random.seed(324591)

geral = pd.DataFrame(columns=['genero', 'predictions', 'real', 'medias'])
tudo_predictions, tudo_real, tudo_medias = [], [], []

# Iterar sobre todos os gêneros em 'musicas_por_genero'
for genero, musicas in musicas_por_genero.items():
    # Variáveis para armazenar as entradas, alvos e predições
    all_inputs, all_targets, all_predictions, all_real, medias = [], [], [], [], []
    print(f"Iniciando o processamento para o gênero: {genero}")
    
    i = 1
    total = len(musicas)
    
    # Iterar sobre os arquivos para processar e fazer predições
    for file in musicas:
        y, sr = librosa.load(file)
        fmin = 50  # Frequência mínima esperada (em Hz)
        fmax = 500  # Frequência máxima esperada (em Hz)
        f0 = librosa.yin(y, fmin=fmin, fmax=fmax, sr=sr)
        semente = np.random.seed(i)
        indices = np.random.randint(1, len(f0) - ((INPUT_DURATION + PREDICTION_DURATION) * SAMPLE_RATE), size=10)
        
        for indice in indices:
            # Sorteio de um ponto aleatório dentro da série
            
            inicio_treino = indice
            n_por_seg = 43
            seg_analise = 5
            fim_treino = inicio_treino + round(n_por_seg*seg_analise)
            inicio_teste = fim_treino + 1
            fim_teste = inicio_teste + round(n_por_seg*seg_analise)

            training_set = f0[inicio_treino:fim_treino]
            test_set = f0[inicio_teste:fim_teste]
            all_real = np.append(all_real, test_set)
            tudo_real = np.append(tudo_real, test_set)

            sc = MinMaxScaler(feature_range=(0, 1))
            training_set = training_set.reshape(-1, 1)
            training_set_scaled = sc.fit_transform(training_set)

            n_steps = INPUT_DURATION * SAMPLE_RATE
            features = 1

            dataset_total = f0
            inputs = dataset_total[len(dataset_total) - len(test_set) - n_steps :]

            inputs = inputs.reshape(-1, 1)
            # Scaling
            inputs = sc.transform(inputs)

            # Split into samples
            X_test, y_test = split_sequence(inputs, n_steps)
            # Reshape
            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], features)
            # Prediction
            predicted_freq = model.predict(X_test)
            # Inverse transform the values
            predicted_freq = sc.inverse_transform(predicted_freq)

            # Adicionar os dados das predições para este arquivo
            all_predictions = np.append(all_predictions, np.array(predicted_freq))
            medias = np.append(medias, np.repeat(np.mean(training_set), len(test_set)))

        print(f"Gênero: {genero} - Música: {i} - Progresso: {i/total*100:.2f}%")
        i += 1

    # Concatenar os dados de todas as músicas do gênero
    tudo_predictions = np.append(tudo_predictions, all_predictions)
    tudo_medias = np.append(tudo_medias, medias)

     # Criar um DataFrame temporário com o gênero e os valores correspondentes
    df_temp = pd.DataFrame({
        'genero': [genero] * len(all_predictions),  # Replicar o gênero
        'predictions': all_predictions,
        'real': all_real,
        'medias': medias
    })

    # Concatenar o DataFrame temporário com o DataFrame geral
    geral = pd.concat([geral, df_temp], ignore_index=True)
    geral.to_csv("tabela_geral.csv", index=False)

    # Calcular o RMSE para o gênero atual
    rmse = calculate_rmse_global(np.array(all_predictions), np.array(all_real))
    mape = mean_absolute_percentage_error(y_true=all_real, y_pred=all_predictions)
    print("Gênero atual:", genero, "\nRMSE:", rmse, "\nMAPE", mape)
    
    # Adicionar o RMSE calculado para o gênero no DataFrame
    results.loc[len(results)] = [genero, rmse_base, rmse, mape]
    

# Exibir o DataFrame com os resultados dos RMSEs por gênero
print(results)
results.to_csv("tabela_final.csv", index=False)


Iniciando o processamento para o gênero: Electronic
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Gênero: Electronic - Música: 1 - Progresso: 0.46%
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Gênero: Electronic - Música: 2 - Progresso: 0.91%
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
7/7 ━━━━━━━━

C:\Users\vitor\AppData\Local\Temp\ipykernel_1660\1095005360.py:172: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  geral = pd.concat([geral, df_temp], ignore_index=True)


Gênero atual: Electronic 
RMSE base: 77.13730851728728 
RMSE ajustado: 80.97613692430144 
MAPE 0.5270569693603798
Iniciando o processamento para o gênero: Experimental
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Gênero: Experimental - Música: 1 - Progresso: 0.50%
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Gênero: Experimental

In [ ]:
# Resultados 1

import pandas as pd

# Criar dicionário com a contagem de arquivos por gênero
contagem_por_genero = {genero: len(arquivos) for genero, arquivos in musicas_por_genero.items()}

# # Transformar em DataFrame e garantir que "Quantidade" seja inteiro
df_contagem = pd.DataFrame(contagem_por_genero.items(), columns=["Gênero", "Quantidade"])
df_contagem["Quantidade"] = df_contagem["Quantidade"].astype(int)  # Forçar tipo inteiro

# # Juntar com rmse_results
rmse_results = rmse_results.merge(df_contagem, on="Gênero", how="left")
rmse_results = rmse_results.merge(mape_results, on="Gênero", how="left")

# # Reordenar colunas para inserir "Quantidade" entre "Gênero" e "RMSE"
colunas_reordenadas = ["Gênero", "Quantidade", "MAPE", "RMSE"] + [col for col in rmse_results.columns if col not in ["Gênero", "Quantidade", "MAPE", "RMSE"]]
rmse_results = rmse_results[colunas_reordenadas]

# # Exibir resultado
print(rmse_results.dtypes)  # Verificar os tipos das colunas
print(rmse_results)

# # Somar todas as quantidades, exceto o NaN na última linha
soma_quantidade = rmse_results["Quantidade"].dropna().sum()

# # Substituir o NaN da última linha pelo valor da soma
rmse_results.at[rmse_results.index[-1], "Quantidade"] = soma_quantidade

# Renomear a coluna "Quantidade" para "n_musicas"
rmse_results = rmse_results.rename(columns={"Quantidade": "n_arquivos"})

# Ordenar segundo a coluna RMSE
# Separar a última linha (aquela que não deve ser ordenada)
ultima_linha = rmse_results.iloc[[-1]]

# Ordenar o restante do DataFrame pelo RMSE (do menor para o maior)
rmse_results_sorted = rmse_results.iloc[:-1].sort_values(by="RMSE")

# Concatenar a última linha com o DataFrame ordenado
rmse_results_final = pd.concat([rmse_results_sorted, ultima_linha])

# Exibir o DataFrame final
print(rmse_results_final)


AttributeError: 'numpy.float64' object has no attribute 'merge'

In [ ]:
# Tabela 2

import pandas as pd

subgeneros = pd.read_csv("C:/Vitor Hugo/TCC/Teste/__pycache__/subgeneros.csv")
print(subgeneros)

    Unnamed: 0  ID Genero Pai     Genero Pai  ID Subgenero      Subgenero
0            1              2  International            46  Latin America
1            2              2  International            77         French
2            3              2  International            79   Reggae - Dub
3            4              2  International            86         Indian
4            5              2  International            92        African
5            6              2  International           102    Middle East
6            7              2  International           117          Polka
7            8              2  International           118         Balkan
8            9              2  International           130         Europe
9           10              2  International           171      Brazilian
10          11              2  International           172  Asia-Far East
11          12              2  International           176        Pacific
12          13              2  Interna